In [1]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm

import warnings
warnings.filterwarnings('ignore')
train_path = 'C:/Users/Administrator/input/hy_round1_train_20200102'
test_path = 'C:/Users/Administrator/input/hy_round1_testA_20200102'
#print(os.path.abspath('.')) 表示返回当前路径

In [2]:
train_files = os.listdir(train_path)
test_files = os.listdir(test_path)
print(len(train_files),len(test_files))


7000 2000


In [3]:
df = pd.read_csv(f'{train_path}/6966.csv')  #字符串前加f表示支持大括号的表示内容
df.head()


,渔船ID,x,y,速度,方向,time,type
0,6966,6.265902e+06,5.279254e+06,0.11,306,1106 23:58:16,围网
1,6966,6.265902e+06,5.279254e+06,0.00,0,1106 23:48:21,围网
2,6966,6.265902e+06,5.279254e+06,0.00,0,1106 23:38:19,围网
3,6966,6.265902e+06,5.279254e+06,0.00,0,1106 23:28:36,围网
4,6966,6.265902e+06,5.279254e+06,0.32,130,1106 23:08:17,围网


In [4]:
ret = []
for file in tqdm(train_files):
    df = pd.read_csv(f'{train_path}/{file}')
    ret.append(df)
train = pd.concat(ret)
train.columns = ['渔船ID','X','Y','速度','方向','time','type']

100%|█████████████████████████████████████| 7000/7000 [00:16<00:00, 428.03it/s]


In [5]:
train.head()

,渔船ID,X,Y,速度,方向,time,type
0,0,6.152038e+06,5.124873e+06,2.59,102,1110 11:58:19,拖网
1,0,6.151230e+06,5.125218e+06,2.70,113,1110 11:48:19,拖网
2,0,6.150421e+06,5.125563e+06,2.70,116,1110 11:38:19,拖网
3,0,6.149612e+06,5.125907e+06,3.29,95,1110 11:28:19,拖网
4,0,6.148803e+06,5.126252e+06,3.18,108,1110 11:18:19,拖网


In [6]:
#时间特征进行提取
def extract_time(df):
    df['time'] = pd.to_datetime(df['time'],format='%m%d %H:%M:%S')  
    df['month'] = df['time'].dt.month
    df['day'] = df['time'].dt.day
    df['hour'] = df['time'].dt.hour
    return df

In [7]:
train['dX'] = 0.00
train['dY'] = 0.00
train['加速度'] = 0.00
train['方向变化'] = 0
extract_time(train)

,渔船ID,X,Y,速度,方向,time,type,dX,dY,加速度,方向变化,month,day,hour
0,0,6.152038e+06,5.124873e+06,2.59,102,1900-11-10 11:58:19,拖网,0.0,0.0,0.0,0,11,10,11
1,0,6.151230e+06,5.125218e+06,2.70,113,1900-11-10 11:48:19,拖网,0.0,0.0,0.0,0,11,10,11
2,0,6.150421e+06,5.125563e+06,2.70,116,1900-11-10 11:38:19,拖网,0.0,0.0,0.0,0,11,10,11
3,0,6.149612e+06,5.125907e+06,3.29,95,1900-11-10 11:28:19,拖网,0.0,0.0,0.0,0,11,10,11
4,0,6.148803e+06,5.126252e+06,3.18,108,1900-11-10 11:18:19,拖网,0.0,0.0,0.0,0,11,10,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
361,999,6.138413e+06,5.162715e+06,0.32,40,1900-10-31 13:09:00,拖网,0.0,0.0,0.0,0,10,31,13
362,999,6.138412e+06,5.162606e+06,0.22,275,1900-10-31 12:48:58,拖网,0.0,0.0,0.0,0,10,31,12
363,999,6.138413e+06,5.162715e+06,0.32,0,1900-10-31 12:28:01,拖网,0.0,0.0,0.0,0,10,31,12
364,999,6.138413e+06,5.162715e+06,0.32,0,1900-10-31 12:18:00,拖网,0.0,0.0,0.0,0,10,31,12


In [ ]:
#新创建一个dX列，用于记录X的每次变化
#total_seconds()方法可以用于计算两个时间之间相差的秒数
#dX、dY都表示10分钟内的X、Y变化
#加速度表示的是10分钟内速度的变化
#方向变化就是基于上一次信息的方向值的差（与时间无关）
for name,df in tqdm(train.groupby('渔船ID')):
    
    for i in range(2,df.shape[0] + 1): 
        df.iloc[-i,7] = ((df.iloc[-i,1] - df.iloc[-i + 1,1]) / ((df.iloc[-i,5] - df.iloc[-i + 1,5]).total_seconds())) * 600
        df.iloc[-i,8] = ((df.iloc[-i,2] - df.iloc[-i + 1,2]) / ((df.iloc[-i,5] - df.iloc[-i + 1,5]).total_seconds())) * 600
        df.iloc[-i,9] = ((df.iloc[-i,3] - df.iloc[-i + 1,3]) / ((df.iloc[-i,5] - df.iloc[-i + 1,5]).total_seconds())) * 600
        df.iloc[-i,10] = df.iloc[-i,4] - df.iloc[-i + 1,4]
    train.loc[train['渔船ID']==name,'dX'] = df.iloc[:,7]
    train.loc[train['渔船ID']==name,'dY'] = df.iloc[:,8]
    train.loc[train['渔船ID']==name,'加速度'] = df.iloc[:,9]
    train.loc[train['渔船ID']==name,'方向变化'] = df.iloc[:,10]

In [8]:
train

,渔船ID,X,Y,速度,方向,time,type,dX,dY,加速度,方向变化,month,day,hour
0,0,6.152038e+06,5.124873e+06,2.59,102,1900-11-10 11:58:19,拖网,808.914660,-344.632139,-0.110000,-11,11,10,11
1,0,6.151230e+06,5.125218e+06,2.70,113,1900-11-10 11:48:19,拖网,808.872353,-344.699932,0.000000,-3,11,10,11
2,0,6.150421e+06,5.125563e+06,2.70,116,1900-11-10 11:38:19,拖网,808.830024,-344.767718,-0.590000,21,11,10,11
3,0,6.149612e+06,5.125907e+06,3.29,95,1900-11-10 11:28:19,拖网,808.787673,-344.835498,0.110000,-13,11,10,11
4,0,6.148803e+06,5.126252e+06,3.18,108,1900-11-10 11:18:19,拖网,805.325842,-562.955816,0.210000,-21,11,10,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
361,999,6.138413e+06,5.162715e+06,0.32,40,1900-10-31 13:09:00,拖网,0.903971,54.482620,0.049917,-235,10,31,13
362,999,6.138412e+06,5.162606e+06,0.22,275,1900-10-31 12:48:58,拖网,-0.864418,-52.098735,-0.047733,275,10,31,12
363,999,6.138413e+06,5.162715e+06,0.32,0,1900-10-31 12:28:01,拖网,0.000000,0.000000,0.000000,0,10,31,12
364,999,6.138413e+06,5.162715e+06,0.32,0,1900-10-31 12:18:00,拖网,0.000000,0.000000,0.209651,-294,10,31,12


In [9]:
#import tables
train.to_csv('C:/Users/Administrator/input/train_new.csv',index=False)



In [4]:
ret = []
for file in tqdm(test_files):
    df = pd.read_csv(f'{test_path}/{file}')
    ret.append(df)
test = pd.concat(ret)
test.columns = ['渔船ID','X','Y','速度','方向','time']



100%|██████████| 2000/2000 [00:04<00:00, 443.75it/s]


In [5]:
test['dX'] = 0.00
test['dY'] = 0.00
test['加速度'] = 0.00
test['方向变化'] = 0
extract_time(test)

,渔船ID,X,Y,速度,方向,time,dX,dY,加速度,方向变化,month,day,hour
0,7000,7.118845e+06,5.918277e+06,0.11,0,1900-11-03 11:54:32,0.0,0.0,0.0,0,11,3,11
1,7000,7.118940e+06,5.918285e+06,0.32,346,1900-11-03 11:44:32,0.0,0.0,0.0,0,11,3,11
2,7000,7.118948e+06,5.918174e+06,0.11,0,1900-11-03 11:34:43,0.0,0.0,0.0,0,11,3,11
3,7000,7.118948e+06,5.918174e+06,0.11,71,1900-11-03 11:14:30,0.0,0.0,0.0,0,11,3,11
4,7000,7.118948e+06,5.918174e+06,0.11,30,1900-11-03 11:04:46,0.0,0.0,0.0,0,11,3,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...
372,8999,6.283750e+06,5.284013e+06,0.00,0,1900-10-28 00:59:02,0.0,0.0,0.0,0,10,28,0
373,8999,6.283750e+06,5.284013e+06,0.00,0,1900-10-28 00:49:26,0.0,0.0,0.0,0,10,28,0
374,8999,6.283750e+06,5.284013e+06,0.05,0,1900-10-28 00:29:04,0.0,0.0,0.0,0,10,28,0
375,8999,6.283750e+06,5.284013e+06,0.05,0,1900-10-28 00:19:04,0.0,0.0,0.0,0,10,28,0


In [ ]:
#新创建一个dX列，用于记录X的每次变化
#total_seconds()方法可以用于计算两个时间之间相差的秒数
#dX、dY都表示10分钟内的X、Y变化
#加速度表示的是10分钟内速度的变化
#方向变化就是基于上一次信息的方向值的差（与时间无关）
for name,df in test.groupby('渔船ID'):
    print(name)
    for i in range(2,df.shape[0] + 1): 
        df.iloc[-i,6] = ((df.iloc[-i,1] - df.iloc[-i + 1,1]) / ((df.iloc[-i,5] - df.iloc[-i + 1,5]).total_seconds())) * 600
        df.iloc[-i,7] = ((df.iloc[-i,2] - df.iloc[-i + 1,2]) / ((df.iloc[-i,5] - df.iloc[-i + 1,5]).total_seconds())) * 600
        df.iloc[-i,8] = ((df.iloc[-i,3] - df.iloc[-i + 1,3]) / ((df.iloc[-i,5] - df.iloc[-i + 1,5]).total_seconds())) * 600
        df.iloc[-i,9] = df.iloc[-i,4] - df.iloc[-i + 1,4]
    test.loc[test['渔船ID']==name,'dX'] = df.iloc[:,6]
    test.loc[test['渔船ID']==name,'dY'] = df.iloc[:,7]
    test.loc[test['渔船ID']==name,'加速度'] = df.iloc[:,8]
    test.loc[test['渔船ID']==name,'方向变化'] = df.iloc[:,9]

In [11]:
test[test['渔船ID']==8001]

,渔船ID,X,Y,速度,方向,time,dX,dY,加速度,方向变化,month,day,hour
0,8001,6.264979e+06,5.251557e+06,0.11,150,1900-10-30 23:56:12,0.000000,0.000000,0.000000,150,10,30,23
1,8001,6.264979e+06,5.251557e+06,0.11,0,1900-10-30 23:46:12,0.000000,0.000000,0.111298,0,10,30,23
2,8001,6.264979e+06,5.251557e+06,0.00,0,1900-10-30 23:36:19,0.000000,0.000000,0.000000,0,10,30,23
3,8001,6.264979e+06,5.251557e+06,0.00,0,1900-10-30 23:26:10,-101.030492,1.062590,-0.320534,0,10,30,23
4,8001,6.265080e+06,5.251556e+06,0.32,0,1900-10-30 23:16:11,100.862108,-1.060819,0.210000,-302,10,30,23
...,...,...,...,...,...,...,...,...,...,...,...,...,...
378,8001,6.265081e+06,5.251666e+06,0.22,143,1900-10-28 00:41:27,0.000000,0.000000,0.000000,143,10,28,0
379,8001,6.265081e+06,5.251666e+06,0.22,0,1900-10-28 00:31:26,0.000000,0.000000,0.224109,-111,10,28,0
380,8001,6.265081e+06,5.251666e+06,0.00,111,1900-10-28 00:21:37,0.000000,0.000000,-0.312195,-14,10,28,0
381,8001,6.265081e+06,5.251666e+06,0.32,125,1900-10-28 00:11:22,0.000000,0.000000,0.100334,7,10,28,0


In [13]:
test.to_csv('C:/Users/Administrator/input/test_new.csv',index=False)

